In [2]:
'''
The particular script gives various orientation to increase the diversity of orientations for training the model. 
However, initially, I am just interested in looking at the planar molecule. '''


import numpy as np
from scipy.spatial import ConvexHull
import scipy

def parse_fchk(file_path):
    """
    Extract atomic numbers and Cartesian coordinates from a Gaussian .fchk file.
    
    Args:
        file_path (str): Path to the .fchk file.
    
    Returns:
        np.ndarray: Array of shape (n, 4) with [x, y, z, atomic_number] for each atom.
    """
    atomic_numbers = []
    coordinates = []
    num_atoms = 0
    
    with open(file_path, 'r') as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            if line.startswith('Number of atoms'):
                num_atoms = int(line.split()[-1])
            elif line.startswith('Atomic numbers'):
                start_line = i + 1
                values = []
                while len(values) < num_atoms and start_line < len(lines):
                    values.extend([int(x) for x in lines[start_line].split()])
                    start_line += 1
                atomic_numbers = values[:num_atoms]
            elif line.startswith('Current cartesian coordinates'):
                start_line = i + 1
                values = []
                while len(values) < 3 * num_atoms and start_line < len(lines):
                    values.extend([float(x) for x in lines[start_line].split()])
                    start_line += 1
                coordinates = values[:3 * num_atoms]
    
    if not atomic_numbers or not coordinates or num_atoms == 0:
        raise ValueError("Failed to parse atomic numbers or coordinates from .fchk file.")
    
    coordinates = np.array(coordinates).reshape(num_atoms, 3)
    xyz = np.column_stack((coordinates, atomic_numbers))
    return xyz

def _convert_elemements(bias_dict):
    """Convert element symbols to atomic numbers or leave as is."""
    elem_map = {
        'H': 1,   'He': 2,  'Li': 3,  'Be': 4,  'B': 5,
        'C': 6,   'N': 7,   'O': 8,   'F': 9,   'Ne': 10
    }
    return {elem_map.get(k, k): v for k, v in bias_dict.items()}

def plane_from_3points(points):
    """Calculate plane equation from 3 points: ax + by + cz + d = 0."""
    p1, p2, p3 = points
    v1 = p2 - p1
    v2 = p3 - p1
    normal = np.cross(v1, v2)
    normal = normal / np.linalg.norm(normal)
    d = -np.dot(normal, p1)
    return np.array([*normal, d])

def plane_from_2points(points):
    """Calculate plane equation from 2 points, assuming z=0 for third direction."""
    p1, p2 = points
    v = p2 - p1
    if abs(v[0]) > abs(v[1]):
        u = np.array([0, 1, 0])
    else:
        u = np.array([1, 0, 0])
    normal = np.cross(v, u)
    normal = normal / np.linalg.norm(normal)
    d = -np.dot(normal, p1)
    return np.array([*normal, d])

def get_convex_hull_eqs(xyz, angle_tolerance=5):
    """Get plane equations from convex hull of molecular coordinates."""
    points = xyz[:, :3]
    try:
        hull = ConvexHull(points)
        eqs = []
        for simplex in hull.simplices:
            pts = points[simplex]
            eq = plane_from_3points(pts)
            eqs.append(eq)
        return np.array(eqs), hull
    except scipy.spatial.qhull.QhullError:
        raise

def find_planar_segments(xyz, eqs, dist_tol=0.1, num_atoms=6):
    """Identify planar segments with at least num_atoms within dist_tol."""

    """Need to adapt this code to detect all the planes which have more than 3 atoms, and sort them according to the most elements"""
    planar_eqs = []
    planar_indices = []
    count = []
    coords = xyz[:, :3]
    for i, eq in enumerate(eqs):
        normal, d = eq[:3], eq[3]
        distances = np.abs(np.dot(coords, normal) + d) / np.linalg.norm(normal)
        in_plane = np.where(distances < dist_tol)[0]
        if len(in_plane) >= num_atoms:
            planar_eqs.append(eq)
            planar_indices.append(i)
            count.append(len(in_plane))
    return np.array(planar_eqs), np.array(planar_indices), np.array(count)

def get_plane_elements(xyz, eqs, dist_tol=0.7):
    """Get elements within dist_tol of each plane."""
    coords = xyz[:, :3]
    elements = xyz[:, -1].astype(int)
    plane_elems = []
    for eq in eqs:
        normal, d = eq[:3], eq[3]
        distances = np.abs(np.dot(coords, normal) + d) / np.linalg.norm(normal)
        in_plane = np.where(distances < dist_tol)[0]
        plane_elems.append(set(elements[in_plane]))
    return plane_elems

def random_unit_vector():
    """Generate a random unit vector in 3D."""
    phi = np.random.uniform(0, 2 * np.pi)
    costheta = np.random.uniform(-1, 1)
    theta = np.arccos(costheta)
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)
    return np.array([x, y, z])

def cart_to_sph(vec):
    """Convert Cartesian vector to spherical coordinates (r, phi, theta)."""
    x, y, z = vec
    r = np.sqrt(x**2 + y**2 + z**2)
    phi = np.arctan2(y, x)
    theta = np.arccos(z / r) if r > 0 else 0
    return r, phi, theta

def zyz_rotation(phi, theta, psi):
    """Generate ZYZ Euler rotation matrix."""
    c1, s1 = np.cos(phi), np.sin(phi)
    c2, s2 = np.cos(theta), np.sin(theta)
    c3, s3 = np.cos(psi), np.sin(psi)
    R1 = np.array([[c1, -s1, 0], [s1, c1, 0], [0, 0, 1]])
    R2 = np.array([[c2, 0, s2], [0, 1, 0], [-s2, 0, c2]])
    R3 = np.array([[c3, -s3, 0], [s3, c3, 0], [0, 0, 1]])
    return R1 @ R2 @ R3

def matrix_to_zyz_angles(R, deg=True):
    """
    Convert a 3x3 rotation matrix to ZYZ Euler angles (phi, theta, psi).
    
    Args:
        R (np.ndarray): 3x3 rotation matrix.
        deg (bool): If True, return angles in degrees; else radians.
    
    Returns:
        tuple: (phi, theta, psi) angles.
    """
    R = np.asarray(R)
    if R.shape != (3, 3):
        raise ValueError("Input must be a 3x3 matrix")
    
    # Extract theta
    theta = np.arccos(np.clip(R[2, 2], -1, 1))
    
    # Check for singularity (theta = 0 or pi)
    if np.abs(np.sin(theta)) < 1e-10:
        # When theta = 0 or pi, phi and psi combine
        phi = 0
        psi = np.arctan2(R[1, 0], R[0, 0])
    else:
        phi = np.arctan2(R[1, 2], R[0, 2])
        psi = np.arctan2(R[2, 1], -R[2, 0])
    
    if deg:
        phi = np.degrees(phi)
        theta = np.degrees(theta)
        psi = np.degrees(psi)
    
    return phi, theta, psi

def apply_zyz_rotation(xyz, phi, theta, psi, deg=True):
    """
    Apply ZYZ Euler rotation to xyz coordinates.
    
    Args:
        xyz (np.ndarray): Array of shape (n, 3) or (n, 4) with coordinates.
        phi, theta, psi (float): ZYZ Euler angles.
        deg (bool): If True, angles are in degrees; else radians.
    
    Returns:
        np.ndarray: Rotated coordinates.
    """
    if deg:
        phi, theta, psi = np.radians([phi, theta, psi])
    R = zyz_rotation(phi, theta, psi)
    coords = xyz[:, :3] if xyz.shape[1] > 3 else xyz
    rotated = coords @ R
    if xyz.shape[1] > 3:
        return np.column_stack((rotated, xyz[:, 3]))
    return rotated

def choose_rotations_bias(xyz, flat=True, angle_tolerance=5,
                         elem_dist_tol=0.7, flat_dist_tol=0.1, flat_num_atoms=6):
    """
    Choose rotation matrices and angles for a molecule based on geometry and element biases.
    
    Args:
        xyz (np.ndarray): Array of shape (n, 4) with x, y, z coordinates and element IDs.
        flat (bool): If True, prioritize planar segments.
        angle_tolerance (float): Minimum angle (degrees) between normal vectors.
        elem_dist_tol (float): Distance tolerance for elements in a plane.
        flat_dist_tol (float): Distance tolerance for planar segments.
        flat_num_atoms (int): Minimum number of atoms in a planar segment.
    
    Returns:
        list: List of (rotation_matrix, angles, source) tuples, where angles are (phi, theta, psi).
    """
    n_vecs = []
    sources = []
    num_atoms = []
    if len(xyz) > 3:
        try:
            eqs, hull = get_convex_hull_eqs(xyz, angle_tolerance=angle_tolerance)
            vertices = hull.vertices
        except scipy.spatial.qhull.QhullError:
            print(f'A problematic molecule encountered.')
            return []
    elif len(xyz) == 3:
        eqs = plane_from_3points(xyz[:, :3])[None]
        vertices = np.array([0, 1, 2])
    elif len(xyz) == 2:
        eqs = plane_from_2points(xyz[:, :3])[None]
        vertices = np.array([0, 1])
    else:
        print(xyz)
        raise RuntimeError('Molecule with less than two atoms.')
    
    if flat:
        planar_seg_eqs, planar_seg_inds, num_atoms = find_planar_segments(xyz, eqs, dist_tol=flat_dist_tol, num_atoms=flat_num_atoms)
        for eq in planar_seg_eqs:
            n_vecs.append(eq[:3])
            sources.append("Planar segment")
        eqs = np.delete(eqs, planar_seg_inds, axis=0)

    results = []
    for vec, source, num_atom in zip(n_vecs, sources, num_atoms):
        _, phi, theta = cart_to_sph(vec)
        psi = 0  # As used in zyz_rotation(-phi, -theta, 0)
        R = zyz_rotation(-phi, -theta, psi)
        angles = (-np.degrees(phi), -np.degrees(theta), np.degrees(psi))  # Convert to degrees
        results.append((R, angles, source, num_atom))

    return results

def process_fchk_to_rotations(fchk_path, **kwargs):
    """
    Process a Gaussian .fchk file and compute rotation matrices and angles.
    
    Args:
        fchk_path (str): Path to the .fchk file.
        **kwargs: Arguments to pass to choose_rotations_bias.
    
    Returns:
        list: List of (rotation_matrix, angles, source) tuples.
    """
    xyz = parse_fchk(fchk_path)
    rotations = choose_rotations_bias(xyz, **kwargs)
    return rotations


In [34]:
# Example usage
if __name__ == "__main__":
    fchk_file = "/scratch/phys/sin/sethih1/data_files/first_group/7923.fchk"
    fchk_file = "/scratch/phys/sin/sethih1/data_files/first_group/10038.fchk"
    fchk_file = "/scratch/phys/sin/sethih1/data_files/first_group/7847.fchk"
    #fchk_file = "/scratch/phys/sin/sethih1/data_files/first_group/10824.fchk"
    try:
        np.random.seed(42)  # For reproducibility
        rotations = process_fchk_to_rotations(
            fchk_file,
            flat=True,
            angle_tolerance=5,
            elem_dist_tol=0.7,
            flat_dist_tol=0.1,
            flat_num_atoms=2
        )

        rotations = sorted(rotations, key = lambda item: item[3], reverse=True)
        print(f"Number of rotations found: {len(rotations)}")
        for i, (rot, angles, source, num_atoms) in enumerate(rotations, 1):
            phi, theta, psi = angles
            print(f"\nRotation {i} (Source: {source}):")
            print(f"Angles (phi, theta, psi) in degrees: ({phi:.6f}, {theta:.6f}, {psi:.6f})")
            print(f"Rotation matrix:\n{rot}")
            print(f"Number of atoms lying in the plane: \n{num_atoms}")
            # Verify by applying angles
            xyz = parse_fchk(fchk_file)
            rotated_xyz = apply_zyz_rotation(xyz, phi, theta, psi, deg=True)
            matrix_rotated = xyz[:, :3] @ rot
            if np.allclose(rotated_xyz[:, :3], matrix_rotated):
                print("Verification: Angles produce the same rotation as the matrix.")
            else:
                print("Warning: Angle-based rotation differs from matrix.")
    except Exception as e:
        print(f"Error processing .fchk file: {e}")

Error processing .fchk file: [Errno 2] No such file or directory: '/scratch/phys/sin/sethih1/data_files/first_group/7847.fchk'


In [35]:
import numpy as np
import py3Dmol
from ase import Atoms
from ase.data import chemical_symbols
from ase.visualize import view
from IPython.display import display

# Assume parse_fchk and apply_zyz_rotation are defined/imported from your module

def molecule_to_py3dmol(xyz, title='Molecule'):
    '''Create and render a py3Dmol view by manually adding atoms.'''
    atom_list = []
    for atom in xyz:
        x, y, z, atomic_num = atom[:4]
        num = int(atomic_num)
        elem = chemical_symbols[num] if num < len(chemical_symbols) else 'X'
        atom_list.append({'elem': elem, 'x': float(x), 'y': float(y), 'z': float(z)})

    view = py3Dmol.view(width=400, height=300)
    view.addAtoms(atom_list)
    view.setStyle({'sphere': {'scaleFactor': 0.3}, 'stick': {}})
    view.setBackgroundColor('0xFFFFFF')
    view.zoomTo()
    # Explicit render for notebook
    view.render()
    return view

if __name__ == '__main__':
    fchk_file = "/scratch/phys/sin/sethih1/data_files/first_group/7923.fchk"
    xyz = parse_fchk(fchk_file)  # N x 4 array [x,y,z,atomic_number]
    print(f"Loaded {xyz.shape[0]} atoms")

    print(xyz)
    # Visualize original molecule
    v_orig = molecule_to_py3dmol(xyz)
    display(v_orig)

    # Compute rotations
    rotations = process_fchk_to_rotations(
        fchk_file,
        flat=True,
        angle_tolerance=5,
        elem_dist_tol=0.7,
        flat_dist_tol=0.1,
        flat_num_atoms=2
    )
    rot, angles, source, num_atoms = sorted(rotations, key=lambda item: item[3], reverse=True)[0]

    # Apply rotation matrix
    coords = xyz[:, :3]
    rotated_coords_matrix = coords.dot(rot)
    rotated_xyz_matrix = np.hstack((rotated_coords_matrix, xyz[:, 3:4]))

    # Visualize rotated molecule
    v_rot = molecule_to_py3dmol(rotated_xyz_matrix)
    display(v_rot)

    # GUI viewer via ASE if desired
    symbols = [chemical_symbols[int(n)] for n in xyz[:, 3]]
    atoms_orig = Atoms(symbols=symbols, positions=coords)
    atoms_rot = Atoms(symbols=symbols, positions=rotated_coords_matrix)
    view(atoms_orig, viewer='ngl')
    view(atoms_rot, viewer='ngl')


Loaded 9 atoms
[[-1.21698362e-03  1.75258870e+00 -1.34792275e-01  8.00000000e+00]
 [ 4.23671684e+00  1.05481489e+00 -1.17251837e-01  8.00000000e+00]
 [-4.23832500e+00  1.05465048e+00 -3.63299847e-02  8.00000000e+00]
 [ 1.26622989e+00 -2.45440653e+00  1.69689847e-01  6.00000000e+00]
 [-1.26236918e+00 -2.45417220e+00  1.97300635e-01  6.00000000e+00]
 [ 2.13743520e+00  2.20434663e-01 -3.79759362e-02  6.00000000e+00]
 [-2.13807393e+00  2.20119078e-01  4.31235502e-03  6.00000000e+00]
 [ 2.58499794e+00 -4.02985041e+00  2.73290302e-01  1.00000000e+00]
 [-2.57855208e+00 -4.02922869e+00  3.31973857e-01  1.00000000e+00]]


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [41]:
rotated_xyz_matrix
xyz = rotated_xyz_matrix

In [42]:
xyz_new = list(zip(np.int64(xyz[:,3]), 0.529177*xyz[:, :3]))
print(xyz_new)

[(np.int64(8), array([-0.92295525,  0.11561149, -0.00124056])), (np.int64(8), array([-0.27668138,  2.29425721, -0.04115381])), (np.int64(8), array([-0.83387523, -2.15517921,  0.04422972])), (np.int64(6), array([ 1.37541723,  0.50197248, -0.0146998 ])), (np.int64(6), array([ 1.20916481, -0.825535  ,  0.01261579])), (np.int64(6), array([ 0.02444723,  1.13678226, -0.02201617])), (np.int64(6), array([-0.25646659, -1.10789537,  0.02182186])), (np.int64(1), array([ 2.29152761,  1.08982988, -0.02949171])), (np.int64(1), array([ 1.95204774, -1.62101004,  0.027508  ]))]


In [43]:
atomic_symbols = {
    1: "H",   # Hydrogen
    2: "He",  # Helium
    3: "Li",  # Lithium
    4: "Be",  # Beryllium
    5: "B",   # Boron
    6: "C",   # Carbon
    7: "N",   # Nitrogen
    8: "O",   # Oxygen
    9: "F",   # Fluorine
    10: "Ne", # Neon
    11: "Na", # Sodium
    12: "Mg", # Magnesium
    13: "Al", # Aluminum
    14: "Si", # Silicon
    15: "P",  # Phosphorus
    16: "S",  # Sulfur
    17: "Cl", # Chlorine
    18: "Ar", # Argon
    19: "K",  # Potassium
    20: "Ca", # Calcium
    21: "Sc", # Scandium
    22: "Ti", # Titanium
    23: "V",  # Vanadium
    24: "Cr", # Chromium
    25: "Mn", # Manganese
    26: "Fe", # Iron
    27: "Co", # Cobalt
    28: "Ni", # Nickel
    29: "Cu", # Copper
    30: "Zn", # Zinc
    31: "Ga", # Gallium
    32: "Ge", # Germanium
    33: "As", # Arsenic
    34: "Se", # Selenium
    35: "Br", # Bromine
    36: "Kr", # Krypton
    37: "Rb", # Rubidium
    38: "Sr", # Strontium
    39: "Y",  # Yttrium
    40: "Zr", # Zirconium
}

t = [(np.int64(7), [ 1.101043, -1.242036,  0.103446]),
 (np.int64(7),[-2.32007 ,  0.064217, -0.308245]),
 (np.int64(6),[-0.942091,  0.054758, -0.094828]),
 (np.int64(6),[-0.196003,  1.241208,  0.001476]),
 (np.int64(6),[-0.222915, -1.157794, -0.032069]),
 (np.int64(6),[1.187871, 1.159192, 0.143263]),
 (np.int64(6),[ 1.795586, -0.09836 ,  0.187512]),
 (np.int64(1),[-0.698585,  2.211788, -0.038846]),
 (np.int64(1),[-0.767477, -2.108442, -0.09927 ]),
 (np.int64(1),[1.791166, 2.066586, 0.220366]),
 (np.int64(1),[ 2.880272, -0.193584,  0.296971]),
 (np.int64(1),[-2.798421, -0.78505 , -0.0213  ]),
 (np.int64(1),[-2.800156,  0.888542,  0.040968])]
t = xyz_new
text = f"{len(t)}\nComment\n"
for i in range(len(t)):
    atom, pos = t[i]
    pos_str = "\t".join(f"{coord:.6f}" for coord in pos)
    text += atomic_symbols[atom] + "\t" +  pos_str + "\n"
print(text)
xyz = text

9
Comment
O	-0.922955	0.115611	-0.001241
O	-0.276681	2.294257	-0.041154
O	-0.833875	-2.155179	0.044230
C	1.375417	0.501972	-0.014700
C	1.209165	-0.825535	0.012616
C	0.024447	1.136782	-0.022016
C	-0.256467	-1.107895	0.021822
H	2.291528	1.089830	-0.029492
H	1.952048	-1.621010	0.027508



In [45]:
view = py3Dmol.view(width=224, height=224)

# Add the molecule model from the XYZ data
view.addModel(xyz, "xyz")

# Set the style: ball-and-stick (spheres for atoms, sticks for bonds)
view.setStyle({'stick': {}, 'sphere': {'scale': 0.3}})

# Adjust the view so that the molecule fills the available space.
# Optionally, you can specify a padding value (e.g., padding=0.2)
view.zoomTo(padding=0.2)

# Render and display the view (in a Jupyter Notebook, for example)
view.show()
display(view)
# Optionally, to capture a static image (note that this works best in a notebook)
png_data_uri = view.png()
print(png_data_uri)
if png_data_uri:
    import base64
    header, encoded = png_data_uri.split(",", 1)
    png_bytes = base64.b64decode(encoded)
    with open("molecule_fitted.png", "wb") as out_file:
        out_file.write(png_bytes)
    print("Saved static image as molecule_fitted.png")
else:
    print("Could not capture image; ensure you're running in a supported environment (like a Jupyter Notebook).")


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

None
Could not capture image; ensure you're running in a supported environment (like a Jupyter Notebook).
